In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score

train=pd.read_excel("input\train.xlsx")
nrow_train = train.shape[0]
y = train['Y']

NUM_BRANDS = 4000
NUM_CATEGORIES = 1000
NAME_MIN_DF = 10
MAX_FEATURES_ITEM_DESCRIPTION = 50000

对非数值项目暂不考虑

In [5]:
col=list(train.columns)
for i in ['TOOL_ID (#1)', 'TOOL_ID (#2)', 'TOOL_ID (#3)', 'Tool (#2)', 'tool (#1)', 'TOOL', 'TOOL (#1)', 'TOOL (#2)',"Tool"]:
    col.remove(i)

In [6]:
X=train[col[2:]]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.3, random_state=0)

d_train = lgb.Dataset(train_X, label=train_y)
d_valid = lgb.Dataset(valid_X, label=valid_y)
watchlist = [d_train, d_valid]

params = {
    'learning_rate': 0.76,
    'application': 'regression_l2',
    'max_depth': 3,
    'num_leaves': 99,
    'verbosity': -1,
    'metric': 'MSE',
    'nthread': 4
}

In [7]:
model = lgb.train(params, train_set=d_train, num_boost_round=7500, valid_sets=watchlist,early_stopping_rounds=500, verbose_eval=500)

Training until validation scores don't improve for 500 rounds.
[500]	training's l2: 4.18589e-28	valid_1's l2: 0.00139778
Early stopping, best iteration is:
[2]	training's l2: 0.00100409	valid_1's l2: 0.000905423


In [8]:
predsL = model.predict(valid_X)